# Softmax 구현하기 (6-1)
---------------------------------------

## tf.nn.softmax(tf.matmul(X,W)+b)로 사용가능

# Cross Entropy : loss function
---------------------------------------------------------

## tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
## optimizer = tf.train.GradienDescentOptimizer(learning_rate=0.1).minimize(cost)

In [2]:
import tensorflow as tf
tf.__version__

'1.14.0'

In [8]:
x_data = [[1,2,1,1,], [2,1,3,2], [3,1,3,4], [4,1,5,5], [1,7,5,5], [1,2,5,6], [1,6,6,6], [1,7,7,7]]
y_data = [[0,0,1], [0,0,1], [0,0,1], [0,1,0], [0,1,0], [0,1,0], [1,0,0], [1,0,0]]

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

0 3.9211247
200 0.5137991
400 0.41191706
600 0.3318944
800 0.2586267
1000 0.2289034
1200 0.20734374
1400 0.18948042
1600 0.17440894
1800 0.16151403
2000 0.15035516


In [11]:
# Testing & one-hot encoding
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.arg_max(a, 1)))

Instructions for updating:
Use `tf.math.argmax` instead
[[2.7982894e-04 9.9972016e-01 1.3331684e-22]] [1]


In [12]:
# Testing & one-hot encoding
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9],
                                           [1, 3, 4, 4],
                                           [1, 1, 0, 1]]})
    print(a, sess.run(tf.arg_max(a, 1)))

[[1.5253702e-08 4.6960354e-15 1.0000000e+00]
 [1.8465047e-04 1.9354977e-05 9.9979597e-01]
 [2.5239941e-01 1.9817170e-02 7.2778344e-01]] [2 2 2]


## Softmax 다듬기 (6-2)

In [14]:
"""
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

## cross entropy cost/loss

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,       # 필수로 넘겨줘야함
                                                labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
"""

'\nlogits = tf.matmul(X, W) + b\nhypothesis = tf.nn.softmax(logits)\n\n## cross entropy cost/loss\n\ncost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,       # 필수로 넘겨줘야함\n                                                labels=Y_one_hot)\ncost = tf.reduce_mean(cost_i)\n'

In [18]:
import numpy as np

xy = np.loadtxt(r'E:\Develop\python\Machine-Learning\DeepLearning codes\tensorflow v.1.15\data\data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
nb_classes = 7

In [22]:
# one_hot and reshape

Y = tf.placeholder(tf.int32, [None, 1])
Y_one_hot = tf.one_hot(Y, nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

In [25]:
## Predicting animal type based on various features

X = tf.placeholder(tf.float32, [None, 16])       # 데이터 특징이 16개

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [26]:
## 학습하기

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:,.3f}\tAcc: {:.2%}".format(step, loss, acc))
            
    pred = sess.run(prediction, feed_dict={X: x_data})
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:     0	Loss: 7.802	Acc: 20.79%
Step:   100	Loss: 0.653	Acc: 78.22%
Step:   200	Loss: 0.411	Acc: 86.14%
Step:   300	Loss: 0.298	Acc: 92.08%
Step:   400	Loss: 0.235	Acc: 93.07%
Step:   500	Loss: 0.194	Acc: 96.04%
Step:   600	Loss: 0.166	Acc: 98.02%
Step:   700	Loss: 0.145	Acc: 98.02%
Step:   800	Loss: 0.128	Acc: 98.02%
Step:   900	Loss: 0.115	Acc: 98.02%
Step:  1000	Loss: 0.104	Acc: 99.01%
Step:  1100	Loss: 0.095	Acc: 100.00%
Step:  1200	Loss: 0.088	Acc: 100.00%
Step:  1300	Loss: 0.081	Acc: 100.00%
Step:  1400	Loss: 0.076	Acc: 100.00%
Step:  1500	Loss: 0.071	Acc: 100.00%
Step:  1600	Loss: 0.067	Acc: 100.00%
Step:  1700	Loss: 0.063	Acc: 100.00%
Step:  1800	Loss: 0.060	Acc: 100.00%
Step:  1900	Loss: 0.057	Acc: 100.00%
Step:  2000	Loss: 0.054	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction